# Burrows-Wheeler transform for nucleotide or amino acid sequences

The Burrows-Wheeler transform is a very popular string/sequence manipulation algorithm at the basis of many bioinformatics tools as well as compression algorithms. The `paleni_bwt` module provides 
1. A function to compute the BW transform from an input sequence `bwTransform(x)`
2. A function to invert the BW transform from an input transformed sequence `bwInverse(y)`

Both functions check input validity and throw either a TypeError or a customized ValueError exception, TerminatorError. 

The compressed archive includes the module itself, a module named test_paleni_bwt.py with unit tests developed with the `unittest` package and this notebook, in which we provide a test of functionalities and a brief explanation of the algorithm.

## 1. Importing the module and documentation 

In [1]:
import paleni_bwt as bwt
import numpy as np

In [2]:
?bwt

In [3]:
?bwt.bwTransform

In [4]:
?bwt.bwInverse

## 2. Design and development 
### 2.1 Compression

Implementation of the Burrows-Wheeler compression transformation, 
originally described in <a href="#1">[1]</a>, via construction of the suffix array, 
as explained in section 2.2 of <a href="#2">[2]</a>.
The BWT of a string is obtained by definining a matrix of sorted 
rotations of the string after adding a terminator character; the last column of this matrix is the transform. In the 
figure below, googol becomes __lo\$oogg__.

<img src="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2705234/bin/btp324f2.jpg" width="350px">

Another way to obtain the transform is by deriving the suffix array of the string, which is an array containing the start position of all suffixes sorted lexicographically ($S[i]$ in the figure above). The BWT of X is then defined as:<br/><br/>

<div align="left"> $$ B[i] = \begin{cases} $, & \mbox{if } S[i] = 0 \\ X[S[i]-1], & \mbox{otherwise} \end{cases} $$ </div>

Which corresponds to the last character of each row.

This package implements the suffix array approach to compute the transform using `numpy` arrays. The suffixes are created with list comprehension and the suffix array is derived with `numpy.argsort`. The array of indices of `B[i]` is computed with `numpy.where` from the suffix array.

### 2.1 Decompression

Implementation of the Burrows-Wheeler decompression as described 
in <a href="#1">[1]</a>, section 4.2. The transformed string y is the last column $L$ of
the sorted matrix of rotations of the original string ($S$). We can 
obtain the first column $F$ by sorting lexicographically the characters 
in $L$. Given how the matrix is built:

<img src="./antitrasf.png" width="500px" align="right" style="margin:10px">

1. $\forall{i} = 0,...,N-1$, $L[i]$ cyclicly precedes $F[i]$ in $S$ 

To invert the transformation, we need to find a vector $T$ that describes 
the correspondence between each character in $L$ and $F$ such that: 
2. $F[T[j]]=L[j]$

Substituting $i=T[j]$, from 1. we obtain:
3. $L[T[j]]$ precedes $F[T[j]]$  

From 2. we obtain:
4. $L[T[j]]$ cyclicly precedes $L[j]$

Starting from the terminator character $\$ = L[i]$, which is the last character of $S$, $L[T[i]]$ precedes it; then, substituting $j = T[i]$, $L[T[j]]$ precedes $L[j]$, and so on; this way, we reconstruct 
    $S$ back to front. See figure on the right.

The function builds an array $P[N]$ and a dictionary $C$. 
- $C[ch]$ is the 
    total number of instances in $L$ of characters 
    preceding $ch$ in the alphabet
- $P[i]$ is the number of instances of 
    character $L[i]$ in the prefix $L[0:i]$

$T$ is defined as $T[i]=P[i]+C[L[i]]$. In fact, the match of $L[i]$ in $F$ can be found without building F by summing up
1. how many characters in total precede $L[i]$ in $L$ (stored in $C[L[i]]$) 
2. how many of the same character precede it in $L[0:i]$ (stored in $P[i]$)

The function finds the alphabet via `numpy.unique` and derives `P` and `C` using the inbuilt `string.count` function to count characters. Starting from the terminator `y[i]`, it finds the index of the preceding character, `T[i]`; then finds the index of the character preceding `y[T[i]]`, and so on. The indexes are stored in the array `decoded_indexes` in order, such that `S = [y[i] for i in decoded_indexes]`. The terminator is omitted from the output.

## 3. Test of functionalities

We test the package on short words to check visually that they work as intended.

In [6]:
print("transform of 'googol':",bwt.bwTransform("googol"))
print("transform of 'acctg':",bwt.bwTransform("acctg"))

transform of 'googol': lo$oogg
transform of 'acctg': g$actc


In [7]:
print("inverse of 'lo$oogg':",bwt.bwInverse("lo$oogg"))
print("inverse of 'g$actc':",bwt.bwInverse("g$actc"))

inverse of 'lo$oogg': googol
inverse of 'g$actc': acctg


Now we test the functions on real biologically relevant sequences. As an example we take the nucleotide sequence of ATP synthase CF1 alpha subunit (NC_035838.1, REGION: 9728..11251) and protein sequence of RNA polymerase beta subunit (YP_009425274.1) from the _Asplenium prolongatum_ chloroplast genome. We use `%%timeit` to get mean and standard deviation over 100 runs 7 times.

In [2]:
nt = "TCACTTACCTTGCAGTAAGAAAATCTCTGTTGATTCTTCAATTGCTTCCTTTAAAAGCGTTTCCGCTTGTTCCGTAAACACTTTTGTGGAACGAGTAATTTCACCAAATTGAGGTTTATTGGTTATTACATACTCACGTAGCTGAATCAAGAATCGTTTGACTTGTTCAACTTCTGATACATCTAGATATCCGTTGACCCCGGTGTAAGTAGTAGCCACCTGTTCCTCTACCGATAATGGAGCTGACTGAGACTGCTTAAGCAGCTCACGCAATCGCTGACCCCTAGCTAATTGATTCTGGGTAGTCTTATCAAGATCGGAGGCAAATTGGGCGAAAGCTTCCAATTCCGCGAATTGGGCTAATTCCAATTTTAATTTGCCAGCCACCTGTTTCATAGCTTTTGTTTGAGCCGCGGAGCCCACTCTAGATACCGAAATACCCACATTTATAGCTGGTCGGATCCCGGCGTTAAATAAATCTGCTGATAAAAAGATTTAGCCATCGGTAATGGAAATAACATTGGTTGGAATGTAAGCTGAGACATCACCCGCTTGAGTCTCAACGATAGGTAAAGCTGTCATGCTACCTTCTCCCAATTGAAGACTTAACTTAGCTGCTCTCTCTAAGAGACGGGAGTGTAGATAAAAAACATCTCCTGGATATGCTTCTCGTCCAGGCGGTCTCCTTAATAGCAGGGACATCTGCCGATAAGCTTGGGCTTGTTTAGATAGGTCATCATGAATAATCGGGGTATGCTGCTTGCGATACATGAAGTATTCAGCTAAAGCTGCTCCCGTGTAGGGAGCAAGATATTGCAGAGTGGCTGGAGAGTTAGCGGTCTCTGACACCACAATTGTATATTCCATGGCACCGCGATCTTGAAAAGTATCTACTACCTGAGCCACAGAAGAAGCTTTCTGACCGATGGCTACGTAAACACATGTAACATTCTGACCTTTCTGATTCAAAATTGTATCTGTAGCTACTGCTGTTTTACCAGTCTGCCTGTCGCCAATAATTAGTTCTCGTTGACCGCGACCTATAGGAATCATGGAATCAATAGCAATAAGGCCTGTTTGTAAAGGTTCGTATACCGAGCGTCTCGAAATAATACCTGGAGCGGGTGATTCAATCAACCTAAACTCAGAAGCTGGTATTTGACCTTTCCCGTCGATAGGTTGCGCCAAAGCATTTACAACACGACCCAAAAACGAGTCACTGACTGGTACTTGAGCAATCTTTCCTGTCGCTTTTACAGAACTACCCTCTTGTATGGTCAAACCATCACCCGTCAGTACGGCCCCAACATTATCCGATTCCAGATTGAGGGCGATTCCTACTGTACCATCCTCAGATTCCACCGATTCGCCAGCCATTACTTTGTTGAGTCCATGAATACGAGCAATCCCATCTCCGACTTAAAGTACGGTACCTATGTTAACAACTTTTACTTCTGAGACATACCCCTCAATTTGCTTACGAATAATGTTACTAATTTCGTCGGGTCGGATGTTTATTACCAT"
m = len(nt)
n_nt = len(np.unique(list(nt)))
print("Computing transform and inverse on a sequence of {} nucleotides ({} letters)".format(m, n_nt))

Computing transform and inverse on a sequence of 1524 nucleotides (4 letters)


In [3]:
%%timeit -n 100
assert bwt.bwInverse(bwt.bwTransform(nt)) == nt

11.5 ms ± 899 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
aa="MTDQIEFPFYNKAIDRVAMRRLIGKLVVCFGIASTTNILDQVKVLGFQQATEASVSLGIDDLLAVPSRGWLVQDAEKQGYVSEQNYRYGSLHAVEKLRQSIEAWYATSECSKREMNSNFRMIDPLNPVHMMSFSGARGSISQVHQLLGMRGLMSDPRGQVIDLPIRKNLREGLSLTEYIISCYGARKGVVDTAVRTSDAGYLTRRLVEVVQHITIRKKDCETTKNVALLNINREKQESVRTISYQKLVGRVLADNVYWDARCIATRNQDISDGLAGNSVASAQPIYVRSPLTRKSIFRICQRRYGRSLAHHDLVELGEAVGIIAGQSVGEPGTQLTSRTFHTGGVFTGDIAEYLRIPFNGLIHFDDKEVHPTRTRHGHPAWMRQNELSVSIKSCSDIFNFVVPAQSLLMIRNGQYVEAQQIIAEVRAKEFPLKERIEKAIYSNLKGEIHSSKFVWHVRDCMGRPIRVVRETGHIRILSGAYSDSDKNGLFHKDRDRVNLKSNFIEKKCDYFGRRENNPVKSKGSPKGIREFGSYPFYLLNGSKTPASNYILSNVKVGRSERPEFVSLLLERQQTKLKTSRFANIDVQLNSILDESDILAINKNIKYQTGVSGILRYGTIEVEPINKSRFFFGGKAEELNFGSWYKVVRRGNFFLIPEEVYTVSEHLSPILVTNNTIAEEGTQLTTAISSKVGGLIQIEKTLANSITIRVLPGYICNPGKATSILTQDFNLIGLGNRTLNRIETKGWVYLQSVALYGRRKTSVLVRPVVKYDVSSHSLGQEIFCVDKPKTQKHAKAQTLLCISHKNGEQTKMMNHAPIQLIQTFLMARRRGNSHKTSLTEKRHLSLINVGINKTLTTFIQINSVVFTPEQKLEINKVSQDLVPIDEPSLTQVDLSNDRNDLVPKYHSITVHSVPEHGAYFLALSPFNIYRNHPLDNSRNNKYEEVIQKHLPRSESDISVPDGSPKKSSLSPKYRSFLQKSLSLKIRDRPVKSRRSRFPPSCPLDFEEVGLLGTPYGICFFSTTLDLGMSRRASLFRNYSMDYLTNTTEHRCWYFIDETKLKMRCPMNPFFTTILVKKPIHLTPKFFNWRNQLIDLGLLISESRSICESNAFTQSGQIVTIHQNYLLVRIGKTLLATRGATPHKISGDTVEEGDTLLTLPYDRLKSGDITQGLPKVEQLPESRPIASIPARIKDLFGRWNRGITKLIGKLWSHFLSAETSMERCQLLLTNEIRKVYESQGVQISERHLEIIIWQLTSRVIASEDGIANVFFPGELVELSQAERMNRVLERPVFYEPIILGMTRASLNTTSFLSEASLQETTRVLAKAALRGRIDRLKGLKENVILGDIVPVGTSSPEIVCQLEVNKQKDFHFAMNGDKKNLNWQAKYALCDYCEKQNIHLIRIIHEGLSRQLLPINPDLKKGLRNIK"
n=len(aa)
n_aa=len(np.unique(list(aa)))
print("Computing transform and inverse on a sequence of {} amino acids ({} letters)".format(n,n_aa))

Computing transform and inverse on a sequence of 1425 amino acids (20 letters)


In [5]:
%%timeit -n 100
assert bwt.bwInverse(bwt.bwTransform(aa)) == aa

9.47 ms ± 585 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Both functions are computed in a reasonable time.

The functions react to to incorrect input by raising exceptions, which you can test in the code block below.

In [8]:
def tests(x, fun, err_type):
    try:
        print("trying to convert",x)
        fun(x)
    except err_type as e:
        print(type(e),e,"\n")
        
tests(5, bwt.bwTransform, TypeError)
tests("lo$oogg", bwt.bwTransform, bwt.TerminatorError)
tests("googol", bwt.bwInverse, bwt.TerminatorError)
tests("lo$oogg$", bwt.bwInverse, bwt.TerminatorError)


trying to convert 5
<class 'TypeError'> The input is not a string 

trying to convert lo$oogg
<class 'paleni_bwt.TerminatorError'> The input cannot contain the terminator character ('$'). Please provide another string. 

trying to convert googol
<class 'paleni_bwt.TerminatorError'> The input does not contain the terminator character ('$'). Please provide a coded string. 

trying to convert lo$oogg$
<class 'paleni_bwt.TerminatorError'> The terminator ('$') cannot be present more than once in the coded string. Please provide another string. 



## References  
<br/>
<div id="1">[1] Burrows, M. & Wheeler, D.J. A Block-Sorting Lossless 
    Data Compression Algorithm (1994). Technical Report, 124, Digital Equipment 
    Corporation.  </div>
<div id="2">[2] Li H, Durbin R. Fast and accurate short read alignment with Burrows-Wheeler transform. Bioinformatics. 2009;25(14):1754-1760. doi:10.1093/bioinformatics/btp324</a><div>
    
http://guanine.evolbio.mpg.de/bwt/bwtDoc.pdf for another explanation of the decoding algorithm